In [ ]:
from datetime import datetime, timedelta
import pytest
import pytz
import logging

import numpy as np
import pandas as pd
from pandas.tseries.frequencies import to_offset
from sqlalchemy import select

from flexmeasures.data.models.time_series import Sensor
from flexmeasures.data.models.planning import Scheduler
from flexmeasures.data.models.planning.exceptions import InfeasibleProblemException
from flexmeasures.data.models.planning.storage import (
    StorageScheduler,
    add_storage_constraints,
    validate_storage_constraints,
    build_device_soc_values,
)
from flexmeasures.data.models.planning.linear_optimization import device_scheduler
from flexmeasures.data.models.planning.tests.utils import check_constraints
from flexmeasures.data.models.planning.utils import initialize_series, initialize_df
from flexmeasures.data.schemas.sensors import TimedEventSchema
from flexmeasures.utils.calculations import (
    apply_stock_changes_and_losses,
    integrate_time_series,
)
from flexmeasures.tests.utils import get_test_sensor
from flexmeasures.utils.unit_utils import convert_units, ur

from flexmeasures.data.models.planning.storage import StorageScheduler

from pyomo.environ import value

TOLERANCE = 0.00001

from flask import current_app
from flexmeasures.app import create

In [ ]:
app = create()

In [ ]:
start = pd.Timestamp("2020-01-01T00:00:00")
end = pd.Timestamp("2020-01-02T00:00:00")
resolution = timedelta(hours=1)
soc_at_start = [0.3, 0.2, 0.3, 0.5, 0.5] * 3
soc_max = [1, 1, 1, 1, 1] * 3
soc_min = [0, 0, 0, 0, 0] * 3

start_datetime = ["2020-01-01 09:00:00"] * 15
target_datetime = ["2020-01-01 17:00:00", "2020-01-01 17:00:00", "2020-01-01 15:00:00", "2020-01-01 16:00:00", "2020-01-01 14:00:00"] * 3
target_value = [1, 1, 1, 1, 1] * 3

market_prices = [
    0.8598, 1.4613, 2430.3887, 3000.1779, 18.6619, 369.3274, 169.8719, 174.2279, 174.2279, 174.2279,
    175.4258, 1.9297, 174.2763, 174.2279, 175.2564, 202.6992, 218.4413, 229.9242, 295.1069, 240.7174,
    249.2479, 238.2732, 229.8395, 216.5779
]

def initialize_combined_constraints(num_devices, soc_at_start, soc_max, soc_min, target_datetime, target_value, start_datetime):
    device_constraints = []
    for i in range(num_devices):
        constraints = initialize_df(StorageScheduler.COLUMNS, start, end, resolution)

        start_time = pd.Timestamp(start_datetime[i]) - timedelta(hours=1)

        constraints["max"] = soc_max[i] - soc_at_start[i]
        constraints["min"] = soc_min[i] - soc_at_start[i]
        constraints["derivative max"] = 1
        constraints["derivative min"] = 0
        constraints["min"][target_datetime[i]] = target_value[i] - soc_at_start[i]
        constraints.loc[:start_time, ["max", "min", "derivative max", "derivative min"]] = 0
        device_constraints.append(constraints)
    return device_constraints

def initialize_combined_commitments(num_devices):
    commitments = []
    for _ in range(num_devices):
        commitment = initialize_df(
            ["quantity", "downwards deviation price", "upwards deviation price", "group"],
            start, end, resolution
        )
        commitment["quantity"] = 0
        commitment["downwards deviation price"] = market_prices
        commitment["upwards deviation price"] = market_prices
        commitment["group"] = list(range(len(commitment)))
        commitments.append(commitment)
    return commitments

def run_simultaneous_scheduler():
    with app.app_context():
        num_devices = len(soc_at_start)
        device_constraints = initialize_combined_constraints(
            num_devices, soc_at_start, soc_max, soc_min, target_datetime, target_value, start_datetime
        )
        commitments = initialize_combined_commitments(num_devices)
        
        ems_constraints = initialize_df(StorageScheduler.COLUMNS, start, end, resolution)
        ems_constraints["derivative max"] = 2
        ems_constraints["derivative min"] = 0
        
        initial_stocks = soc_at_start
        _, _, results, model = device_scheduler(
            device_constraints, ems_constraints, commitments=commitments, initial_stock=initial_stocks
        )
        
        all_schedules = []
        combined_ems_schedule = [0] * len(market_prices)  

        individual_costs = []
        
        for i in range(num_devices):
            schedule = initialize_series(
                data=[model.ems_power[i, j].value for j in model.j],
                start=start, end=end, resolution=resolution
            )
            all_schedules.append(schedule)
            
            combined_ems_schedule = [combined_ems_schedule[j] + schedule[j] for j in range(len(schedule))]

            costs = sum(schedule[j] * market_prices[j] for j in range(len(market_prices)))
            individual_costs.append((i, costs))
        
        total_costs = sum(cost for _, cost in individual_costs)  

        return all_schedules, combined_ems_schedule, results, individual_costs, total_costs

schedules, combined_ems_schedule, results, individual_costs, total_costs = run_simultaneous_scheduler()

print("\n=== Schedules for All Devices ===")
for i, schedule in enumerate(schedules):
    print(f"\nDevice {i} Schedule:")
    print(f"Time Steps | Power (kW)")
    print("="*30)
    for time, power in zip(range(len(schedule)), schedule): 
        print(f"{start + timedelta(hours=time)} | {power:.2f}")
    
print("\n=== Combined EMS Schedule (All Devices Together) ===")
print(f"Time Steps | Combined Power (kW)")
print("="*30)
for time, power in enumerate(combined_ems_schedule):
    print(f"{start + timedelta(hours=time)} | {power:.2f}")

print("\n=== Individual Costs per Device ===")
for device, cost in individual_costs:
    print(f"Device {device}: {cost:.2f} currency units")

print("\n=== Total Costs ===")
print(f"Total Costs: {total_costs:.2f} currency units")




In [ ]:
def load_market_prices(excel_path, sheet_name=0):
    data = pd.read_excel(excel_path, sheet_name=sheet_name)
    data["datetime"] = pd.to_datetime(data["datetime"]) 
    data.set_index("datetime", inplace=True)
    return data["price"]  # 

def run_yearly_scheduler(market_prices, start_date, end_date):
    with app.app_context():
        daily_costs = {}
        current_date = pd.Timestamp(start_date)
        end_date = pd.Timestamp(end_date)
        
        while current_date < end_date:
            day_start = current_date
            day_end = current_date + timedelta(days=1)
            
            daily_prices = market_prices[day_start:day_end]
            if len(daily_prices) > 24:
                daily_prices = daily_prices.iloc[:24] 
            elif len(daily_prices) < 24:
                daily_prices = daily_prices.append(pd.Series([daily_prices.iloc[-1]] * (24 - len(daily_prices))),
                                                ignore_index=True)

            
            # Update global variables for the day
            start = day_start
            end = day_end
            resolution = timedelta(hours=1)
            
            # Run the scheduler for the day
            num_devices = len(soc_at_start)
            device_constraints = initialize_combined_constraints(
                num_devices, soc_at_start, soc_max, soc_min, target_datetime, target_value, start_datetime
            )
            commitments = initialize_combined_commitments(num_devices, daily_prices)
            
            ems_constraints = initialize_df(StorageScheduler.COLUMNS, start, end, resolution)
            ems_constraints["derivative max"] = 1.7
            ems_constraints["derivative min"] = 0
            
            # Simulate the devices
            initial_stocks = soc_at_start
            _, _, results, model = device_scheduler(
                device_constraints, ems_constraints, commitments=commitments, initial_stock=initial_stocks
            )
            
            total_daily_cost = 0
            
            for i in range(num_devices):
                schedule = initialize_series(
                    data=[model.ems_power[i, j].value for j in model.j],
                    start=start, end=end, resolution=resolution
                )
                daily_cost = sum(schedule[j] * daily_prices[j] for j in range(len(schedule)))
                total_daily_cost += daily_cost
            
            daily_costs[current_date.strftime("%Y-%m-%d")] = total_daily_cost
            
            current_date += timedelta(days=1)
        
        return daily_costs

def initialize_combined_commitments(num_devices, daily_prices):
    commitments = []
    for _ in range(num_devices):
        commitment = initialize_df(
            ["quantity", "downwards deviation price", "upwards deviation price", "group"],
            start, end, resolution
        )
        commitment["quantity"] = 0
        commitment["downwards deviation price"] = daily_prices.values
        commitment["upwards deviation price"] = daily_prices.values
        commitment["group"] = list(range(len(commitment)))
        commitments.append(commitment)
    return commitments

excel_path = "sensor_58.xlsx" 
market_prices = load_market_prices(excel_path)
start_date = "2023-01-01"
end_date = "2023-12-31"

daily_expenses = run_yearly_scheduler(market_prices, start_date, end_date)

print("\n=== Daily Total Expenses ===")
for day, cost in daily_expenses.items():
    print(f"{day}: {cost:.2f} currency units")


In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

daily_expenses_df = pd.DataFrame(list(daily_expenses.items()), columns=["Date", "Total Expenses"])
daily_expenses_df["Date"] = pd.to_datetime(daily_expenses_df["Date"])  
daily_expenses_df.set_index("Date", inplace=True)  

plt.figure(figsize=(10, 6))
plt.plot(daily_expenses_df.index, daily_expenses_df["Total Expenses"], label="Total Daily Expenses", color="b")
plt.title("Total Daily Expenses Over the Year")
plt.xlabel("Date")
plt.ylabel("Total Expenses (currency units)")
plt.grid(True)
plt.xticks(rotation=45)  
plt.tight_layout() 
plt.show()
